# Task 2 (Prototype Price Model)

You should write a function that is able to use the data you created previously to price the contract. The client may want to choose multiple dates to inject and withdraw a set amount of gas, so your approach should generalize the explanation from before. Consider all the cash flows involved in the product.

The input parameters that should be taken into account for pricing are:
- Injection dates. 
- Withdrawal dates.
- The prices at which the commodity can be purchased/sold on those dates.
- The rate at which the gas can be injected/withdrawn.
- The maximum volume that can be stored.
- Storage costs.

Write a function that takes these inputs and gives back the value of the contract. You can assume there is no transport delay and that interest rates are zero. Market holidays, weekends, and bank holidays need not be accounted for. Test your code by selecting a few sample inputs.

Steps to Calculate the Value of the Contract
1. Go through every date
1a. If it is an injection date, calculate how much it will cost (cost to buy the gas and cost to inject gas)
1b. If it is a withdrawal date, calculate how much cash is made (rate times the price it is sold at minus the cost to withdraw the gas)
2. Once all dates have been calculated, calculate the total storage cost (number of months between the last date of withdrawal and the first day of injections)
3. Subtrat the buy costs and storage costs from the total cash made to get your final value

In [2]:
import math
from datetime import date

def price_contract(in_dates, in_prices, out_dates, out_prices, rate, storage_cost_rate, total_vol, injection_withdrawal_cost_rate):
    #`in_dates`: A list of dates on which the gas is being injected into the storage facility.
    #`in_prices`: A list of prices of gas on each of the injection dates.
    #`out_dates`: A list of dates on which the gas is being withdrawn from the storage facility.
    #`out_prices`: A list of prices of gas on each of the withdrawal dates.
    #`rate`: The rate of gas in cubic feet per day.
    #`storage_cost_rate`: A fixed monthly fee to store the gas
    #`total_vol`: The total volume of gas in cubic feet that can be stored.
    #`injection_withdrawal_cost_rate`: The injection/withdrawal cost of gas in dollars per cubic foot.
    
    cash_in = 0
    volume = 0
    buy_cost = 0

    #get all dates
    all_dates = set(in_dates+out_dates)
    all_dates = sorted(all_dates)

    #iterate through all dates
    for date in all_dates:
        
        #if injection
        if date in in_dates:
            #if sufficient storage, add on buy costs
            if volume <= total_vol - rate:
                volume +=rate
                buy_cost += rate * in_prices[in_dates.index(date)]
                buy_cost += rate * injection_withdrawal_cost_rate
                print(f"Injected Gas on {date} at a Price of {in_prices[in_dates.index(date)]}")
            else:
                print("Insufficient storage to buy more gas")

        #if withdrawal
        elif date in out_dates:
            #if sufficient volume remaining, add on cash made
            if volume >= rate:
                volume -= rate
                cash_in += rate * out_prices[out_dates.index(date)]
                cash_in -= rate * injection_withdrawal_cost_rate
                print(f"Withdrew Gas on {date} at a Price of {out_prices[out_dates.index(date)]}")
            else:
                print("Insufficient volume to withdraw more gas")
    
    #calculate total storage costs
    total_storage_cost = math.ceil((max(out_dates)-min(in_dates)).days // 30) * storage_cost_rate
    return cash_in - buy_cost - total_storage_cost

In [3]:
# Example usage of price_contract()
in_dates = [date(2022, 1, 1), date(2022, 2, 1), date(2022, 2, 21), date(2022, 4, 1)] #injection dates
in_prices = [20, 21, 20.5, 22]#prices on the injection days
out_dates = [date(2022, 1, 27), date(2022, 2, 15), date(2022, 3, 20), date(2022, 6, 1)] # extraction dates
out_prices = [23, 19, 21, 25] # prices on the extraction days
rate = 100000  # rate of gas in cubic feet per day
storage_cost_rate = 10000  # total volume in cubic feet
injection_withdrawal_cost_rate = 0.0005  # $/cf
max_storage_volume = 500000 # maximum storage capacity of the storage facility
result = price_contract(in_dates, in_prices, out_dates, out_prices, rate, storage_cost_rate, max_storage_volume, injection_withdrawal_cost_rate)
print()
print(f"The value of the contract is: ${result}")

Injected Gas on 2022-01-01 at a Price of 20
Withdrew Gas on 2022-01-27 at a Price of 23
Injected Gas on 2022-02-01 at a Price of 21
Withdrew Gas on 2022-02-15 at a Price of 19
Injected Gas on 2022-02-21 at a Price of 20.5
Withdrew Gas on 2022-03-20 at a Price of 21
Injected Gas on 2022-04-01 at a Price of 22
Withdrew Gas on 2022-06-01 at a Price of 25

The value of the contract is: $399600.0
